In [ ]:
# Setting auto reloading for imported modules
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import missingno as msno
from auxiliary_functions import Utils

In [ ]:
prep_utils = Utils()

In [ ]:
ga_sessions_path = '../data/skillbox_diploma_main_dataset_sberautopodpiska/ga_sessions.csv'
sessions_df = pd.read_csv(ga_sessions_path, low_memory=False)
sessions_df.head()

In [ ]:
ga_hits_path = '../data/skillbox_diploma_main_dataset_sberautopodpiska/ga_hits-002.csv'
hits_df = pd.read_csv(ga_hits_path, low_memory=False)
hits_df.head()

**Data preprocessing**
<br>
*Duplicates*

In [ ]:
prep_utils.print_basic_stats(sessions_df, level='shape')

In [ ]:
sessions_df.describe(include='all')

In [ ]:
hits_df.describe(include='all')

In [ ]:
prep_utils.check_duplicates(sessions_df)
prep_utils.check_duplicates(hits_df)

**Data preprocessing**
<br>
*Handle Missing Values*
<br>
*Sessions dataframe*

In [ ]:
msno.matrix(sessions_df);

In [ ]:
sessions_df_clean = sessions_df.copy()

In [ ]:
prep_utils.missing_values_percentage(sessions_df_clean)

In [ ]:
sessions_df_clean[sessions_df_clean.device_model.notna()]

In [ ]:
sessions_df_clean.device_model.value_counts()

In [ ]:
sessions_df_clean = sessions_df_clean.drop(columns=['device_model'], axis=1)
prep_utils.missing_values_percentage(sessions_df_clean)

In [ ]:
sessions_df_clean[sessions_df_clean.utm_source.isna()].head(20)

In [ ]:
sessions_df_clean[(sessions_df_clean.utm_source.isna()) & (sessions_df_clean.visit_number == 1)]

In [ ]:
sessions_df_clean[sessions_df_clean.utm_source.notna()].head(10)

In [ ]:
sessions_df_clean.utm_source = sessions_df_clean.utm_source.fillna('(not set)')
prep_utils.missing_values_percentage(sessions_df_clean)

In [ ]:
sessions_df_clean[sessions_df_clean.device_os.isna()].head()

In [ ]:
sessions_df_clean.device_os.value_counts()

In [ ]:
brands_w_missing_os = sessions_df_clean[sessions_df_clean.device_os.isna()].device_brand.value_counts().to_dict()
brands_w_missing_os

In [ ]:
sessions_df_clean.loc[(sessions_df_clean.device_os.isna()) & (sessions_df_clean.device_brand == 'Apple'), 'device_os'] = 'iOS'

In [ ]:
android_based = ['Samsung', 'Xiaomi', 'Huawei', 'Realme']
sessions_df_clean.loc[(sessions_df_clean.device_os.isna()) & (sessions_df_clean.device_brand.isin(android_based)), 'device_os'] = 'Android'

In [ ]:
sessions_df_clean.device_os = sessions_df_clean.device_os.fillna('(not set)')

In [ ]:
prep_utils.missing_values_percentage(sessions_df_clean)

In [ ]:
sessions_df_clean[sessions_df_clean.utm_keyword.isna()].head(20)

In [ ]:
sessions_df_clean[sessions_df_clean.utm_keyword.notna()].head(20)

In [ ]:
utm_keyword_values = sessions_df_clean.utm_keyword.value_counts(dropna=False).to_dict()
utm_keyword_values

In [ ]:
sessions_df_clean.utm_keyword = sessions_df_clean.utm_keyword.fillna('(not set)')
prep_utils.missing_values_percentage(sessions_df_clean)

In [ ]:
sessions_df_clean[sessions_df_clean.utm_adcontent.isna()].head(20)

In [ ]:
sessions_df_clean.utm_adcontent = sessions_df_clean.utm_adcontent.fillna('(not set)')
sessions_df_clean.utm_campaign = sessions_df_clean.utm_campaign.fillna('(not set)')
prep_utils.missing_values_percentage(sessions_df_clean)

In [ ]:
sessions_df_clean[sessions_df_clean.device_brand.isna()].head(20)

In [ ]:
sessions_df_clean.device_brand = sessions_df_clean.device_brand.fillna('(not set)')
prep_utils.missing_values_percentage(sessions_df_clean)

**Data Preparation**
<br>
*Hits dataframe*

In [ ]:
msno.matrix(hits_df);

In [ ]:
hits_df.head()

In [ ]:
prep_utils.missing_values_percentage(hits_df)

In [ ]:
target_events = ['sub_car_claim_click', 'sub_car_claim_submit_click',
'sub_open_dialog_click', 'sub_custom_question_submit_click',
'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success',
'sub_car_request_submit_click']

hits_df['target_event'] = hits_df.event_action.apply(lambda x: 1 if x in target_events else 0)
hits_df['CR'] = hits_df.groupby('session_id')['target_event'].transform('max')
hits_df['CR'] = hits_df['CR'].astype('int')
hits_df.head()

In [ ]:
hits_df.CR.value_counts()

In [ ]:
unique_hits_cr = hits_df[['session_id', 'CR']].drop_duplicates(subset='session_id', keep='first')
sessions_w_cr = sessions_df_clean.merge(unique_hits_cr, on='session_id', how='inner')
sessions_w_cr.head()

In [ ]:
sessions_w_cr[sessions_w_cr.CR == 1].head()

In [ ]:
prep_utils.print_basic_stats(df=sessions_w_cr, level='shape')

**Data Preparation**
<br>
*Data Cleaning*
<br>
*Removing Irrelevant Features*

In [ ]:
df = sessions_w_cr.copy()

Task specified which columns must be passed to model, so other irrelevant columns will be dropped.

In [ ]:
columns_to_drop = ['session_id', 'client_id', 'visit_date', 'visit_time', 'visit_number']
df = df.drop(columns=columns_to_drop)
df.head()

In [ ]:
df.to_csv('../data/clean_data/clean_sessions_with_cr.csv', index=False)